In [4]:
import pandas as pd
import numpy as np
import re



In [5]:
df = pd.read_csv('/Users/sreyaathma/Downloads/PinjobsDS_coupons_ner.csv')
df.head()

,"Save $2.00 ONE Downy Liquid Fabric Conditioner 72 ld or larger (includes Downy Odor Protect 48 oz or larger OR Downy Wrinkle Guard 40 oz or larger OR Downy Nature Blends 67 oz or larger) OR Bounce/Downy Sheets 130 ct or larger (includes Bounce/Downy Wrinkle Guard 80 ct or larger) OR In Wash Scent Boosters 8.6 oz or larger (includes Downy Unstopables, Fresh Protect, Odor Protect, and Infusions) (excludes Downy Libre Enjuague, Gain Fireworks, and trial/travel size)."
0,Save $2.00 ONE Tide PODS OR Tide Power PODS (e...
1,Save $2.00 ONE Tide Laundry Detergent (exclude...
2,SAVE $1.00 ON TWO when you buy TWO BOXES (8.9 ...
3,$3.00 OFF when you purchase any THREE (3) Pepp...
4,SAVE $1.11 when you buy any ONE (1) Familly Si...


In [6]:
df = pd.DataFrame(np.vstack([df.columns, df]))
df = df.rename(columns={0: "OfferDetails"})
df.head()

,OfferDetails
0,Save $2.00 ONE Downy Liquid Fabric Conditioner...
1,Save $2.00 ONE Tide PODS OR Tide Power PODS (e...
2,Save $2.00 ONE Tide Laundry Detergent (exclude...
3,SAVE $1.00 ON TWO when you buy TWO BOXES (8.9 ...
4,$3.00 OFF when you purchase any THREE (3) Pepp...


In [7]:
def FaceValue_using_regex(df_text):
  Face_Value = []
  regex = r"[$¢]\s*[.,\d]+|[.,\d]+\s*[¢$]"
  for i in df_text:
    Face_Value.append(re.findall(regex, i, re.I))
  
  return Face_Value

In [8]:
df['FaceValue'] = [','.join(map(str, l)) for l in FaceValue_using_regex(df['OfferDetails'])]

In [9]:
df.head()

,OfferDetails,FaceValue
0,Save $2.00 ONE Downy Liquid Fabric Conditioner...,$2.00
1,Save $2.00 ONE Tide PODS OR Tide Power PODS (e...,$2.00
2,Save $2.00 ONE Tide Laundry Detergent (exclude...,$2.00
3,SAVE $1.00 ON TWO when you buy TWO BOXES (8.9 ...,$1.00
4,$3.00 OFF when you purchase any THREE (3) Pepp...,$3.00


In [10]:
rows_with_nan = [index for index, row in df.iterrows() if row.isnull().any()]
print("Total number of missing FaceValue are:",len(rows_with_nan))

Total number of missing FaceValue are: 0


In [11]:
import nltk
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree

In [18]:
avoid=['OR','ONE','TWO','THREE','ON','BOXES','OZ','LARGE','SIZE','OFF','Size','SAVE','Save','Any','SMALL','Eight','TO','Includes','FREE','Max',\
  'Value','F','Only','GET','ONLY','700-ml','Limit','FOUR','SELECT','BUY','EXCLUDES','Studio','Buy']

def seperater(chunked):
    sub_trees=[]
    other=[]
    for i in chunked:
        if type(i) == Tree:
            t=" ".join([token for token, pos in i.leaves() if token not in avoid])
            if len(t.split())>=1:
                sub_trees.append(t)
        else :
            other.append(i)
    return sub_trees,other

def product_names(text):
    out=[]
    collect=[]
    chunked= nltk.ne_chunk(pos_tag(word_tokenize(text)), binary=True)
    out,remain=seperater(chunked)   
    for i,data in enumerate(remain):
        if (data[1]=="NNP" and data[0] not in avoid) and i<=len(remain)-2 and remain[i+1][1]=='NNP' :
            collect.append(data[0])
        elif data[1]=="NNP" and data[0] in avoid:
            t=" ".join(collect)
            if len(t.split())>=1:
                out.append(t)
                collect=[]
        elif data[1]=="NNP" and i<=(len(remain)-2) and remain[i+1][1]!='NNP' and data[0] not in avoid:
            collect.append(data[0])
            t=" ".join(collect)
            if len(t.split())>=1:
                out.append(t)
                collect=[]
        elif (i)==(len(remain)-1) and data[1]=="NNP" and data[0] not in avoid :
            collect.append(data[0])
            t=" ".join(collect)
            if len(t.split())>=1:
                out.append(t)
                collect=[]
    return out

In [30]:
input1 = df['OfferDetails'].loc[2]
input1


'Save $2.00 ONE Tide Laundry Detergent (excludes Tide Purclean Laundry Detergent, Studio by Tide Laundry Detergent, Tide PODS, Tide Rescue, Tide Simply Laundry Detergent, Tide Simply PODS, Tide Detergent 10 oz and trial/travel size).'

In [31]:
df['Product'] = df['OfferDetails'].apply(lambda x: input1(x))

TypeError: 'str' object is not callable